In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output
import re
import base64
import os
# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from CRUD import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name. NOTE: You will
# likely need more variables for your constructor to handle the hostname and port of the MongoDB
# server, and the database and collection names

username = "aacuser"
password = "SNHU1234"
shelter = AnimalShelter(username, password)


# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

LOGO_PATH = "/home/sunghoyoon_snhu/Grazioso Salvare Logo.png"
def encode_image(path):
    with open(path, "rb") as f:
        return "data:image/png;base64," + base64.b64encode(f.read()).decode()

logo_src = encode_image(LOGO_PATH)

# Header bar with logo + title + your unique identifier
header_bar = html.Div(
    [
        html.Img(
            id="gs-logo",
            src=logo_src,
            alt="Grazioso Salvare",
            style={"height": "90px", "width": "auto"}
        ),
        html.Div(
            [
                html.H1(
                    "Grazioso Salvare Rescue Dashboard",
                    style={"margin": "0 0 6px 0", "fontSize": "28px"}
                ),
                # ✅ Unique identifier with your name
                html.Div(
                    "Analyst: Sungho Yoon",
                    id="unique-identifier",
                    style={"opacity": 0.8}
                ),
            ],
            style={"display": "flex", "flexDirection": "column"}
        ),
    ],
    style={
        "display": "flex",
        "alignItems": "center",
        "gap": "16px",
        "padding": "12px 16px",
        "borderBottom": "1px solid #eaeaea",
        "backgroundColor": "white",
    },
)
app.layout = html.Div([
    header_bar,
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Hr(),
    html.Div([
    html.Label("Rescue Type Filter"),
    dcc.RadioItems(
        id="rescue-filter",
        options=[
            {"label": "Water Rescue", "value": "water"},
            {"label": "Mountain / Wilderness Rescue", "value": "wilderness"},
            {"label": "Disaster / Individual Tracking", "value": "disaster"},
            {"label": "Reset (All)", "value": "reset"},
        ],
        value="reset",
        inline=True,
        persistence=True
    )
], style={"margin": "8px 0 16px"}),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        #FIXME: Set up the features for your interactive data table to make it user-friendly for your client
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="single",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[0],
        page_action="native",
        page_current=0,
        page_size=10

    ),
    html.Br(),
     html.Hr(),
    html.Div([
        dcc.Graph(
            id='pie-breed',
            style={'width': '40%', 'height': '500px'}
        ),
        html.Div(
            id='map-id',  # keep same id so your existing map callback continues to work
            style={'width': '55%'}
        )
    ], style={
        'display': 'flex',
        'flexDirection': 'row',
        'justifyContent': 'space-between',
        'gap': 12,
        'flexWrap': 'wrap'  # responsive on small screens
        }
    ),
])

#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it

def build_breed_pie_from_view(viewData, top_n=10):
    """
    Pie of breed types (top N). Uses current DataTable view if available.
    Groups all remaining breeds into 'Other' so the pie sums to 100%.
    """
    # Prefer the table's visible rows; fall back to full df
    if viewData is not None:
        dff = pd.DataFrame(viewData)
    else:
        dff = df.copy() if isinstance(df, pd.DataFrame) else pd.DataFrame()

    if dff.empty or 'breed' not in dff.columns:
        return px.pie(pd.DataFrame({'breed_primary': [], 'count': []}),
                      names='breed_primary', values='count',
                      title=f'Top {top_n} Breeds (current filter)')

    ser = dff['breed'].dropna().astype(str)
    # Normalize to primary breed: take left of '/', remove trailing ' Mix'
    primary = (
        ser.str.split('/').str[0]
           .str.replace(r'\s+Mix$', '', regex=True)
           .str.strip()
    )

    counts = primary.value_counts()
    if counts.empty:
        return px.pie(pd.DataFrame({'breed_primary': [], 'count': []}),
                      names='breed_primary', values='count',
                      title=f'Top {top_n} Breeds (current filter)')

    # Keep top N; group the rest as 'Other' (so slices sum to 100%)
    if len(counts) > top_n:
        top = counts.head(top_n)
        other_sum = counts.iloc[top_n:].sum()
        counts_plot = pd.concat([top, pd.Series({'Other': other_sum})])
    else:
        counts_plot = counts

    counts_df = counts_plot.reset_index()
    counts_df.columns = ['breed_primary', 'count']

    fig = px.pie(counts_df, names='breed_primary', values='count',
                 title=f"Top {min(top_n, len(counts))} Breeds (current filter)")
    fig.update_traces(textinfo='percent',
                      hovertemplate='%{label}: %{percent} (%{value})')
    fig.update_layout(margin=dict(t=60, r=20, b=20, l=20))
    return fig
@app.callback(
    Output('pie-breed', 'figure'),
    [Input('datatable-id', 'derived_virtual_data'),
     Input('rescue-filter', 'value')]
)
def update_breed_pie(viewData, _mode):
    return build_breed_pie_from_view(viewData)
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]
def _or_regex(field, names):
    """
    Build an $or clause that matches any of the names anywhere in the field (case-insensitive).
    Example: {"$or": [{"breed": /German Shepherd/i}, {"breed": /Rottweiler/i}, ...]}
    """
    return {"$or": [{field: re.compile(n, re.IGNORECASE)} for n in names]}

def rescue_query(mode: str) -> dict:
    """
    Grazioso Salvare rescue profiles (robust, case-insensitive; matches '... Mix' and 'Breed/Other').
    Uses 'sex_upon_outcome' and 'age_upon_outcome_in_weeks' as in the AAC dataset.
    """
    mode = (mode or "reset").strip().lower()
    base = {"animal_type": re.compile(r"^dog$", re.IGNORECASE)}

    if mode == "water":
        # Labrador Retriever, Chesapeake Bay Retriever, Newfoundland | Intact Female | 26–156 weeks
        q = {
            **base,
            **_or_regex("breed", ["Labrador Retriever", "Chesapeake Bay Retriever", "Newfoundland"]),
            "sex_upon_outcome": re.compile(r"^intact\s+female$", re.IGNORECASE),
            "age_upon_outcome_in_weeks": {"$exists": True, "$gte": 26, "$lte": 156}
        }
        return q

    if mode == "wilderness":
        # German Shepherd, Alaskan Malamute, Old English Sheepdog, Siberian Husky, Rottweiler
        # Intact Male | 26–156 weeks
        q = {
            **base,
            **_or_regex("breed", [
                "German Shepherd", "Alaskan Malamute", "Old English Sheepdog",
                "Siberian Husky", "Rottweiler"
            ]),
            "sex_upon_outcome": re.compile(r"^intact\s+male$", re.IGNORECASE),
            "age_upon_outcome_in_weeks": {"$exists": True, "$gte": 26, "$lte": 156}
        }
        return q

    if mode == "disaster":
        # Doberman Pinscher, German Shepherd, Golden Retriever, Bloodhound, Rottweiler
        # Intact Male | 20–300 weeks
        q = {
            **base,
            **_or_regex("breed", [
                "Doberman Pinscher", "German Shepherd", "Golden Retriever",
                "Bloodhound", "Rottweiler"
            ]),
            "sex_upon_outcome": re.compile(r"^intact\s+male$", re.IGNORECASE),
            "age_upon_outcome_in_weeks": {"$exists": True, "$gte": 20, "$lte": 300}
        }
        return q

    # reset (all)
    return {}

def make_json_safe(records):
    """
    Convert Mongo results to DataTable-friendly dicts:
    - drop/str() any non-JSON types (e.g., ObjectId)
    - ensure the row has the same keys as your original df.columns
    """
    safe = []
    cols = list(df.columns)  # keep DataTable column order stable
    for r in records or []:
        row = {}
        for c in cols:
            v = r.get(c, None)
            # Make values JSON-safe
            if hasattr(v, "item"):
                try:
                    v = v.item()
                except Exception:
                    v = str(v)
            elif hasattr(v, "isoformat"):
                try:
                    v = v.isoformat()
                except Exception:
                    v = str(v)
            elif v is not None and not isinstance(v, (str, int, float, bool)):
                try:
                    v = str(v)
                except Exception:
                    v = None
            row[c] = v
        safe.append(row)
    return safe
@app.callback(
    Output('datatable-id', 'data'),
    [Input('rescue-filter', 'value')]
)
def apply_rescue_filter(mode):
    q = rescue_query(mode)
    try:
        # Use your CRUD module from Step 3 (AnimalShelter)
        # If your read returns a cursor, wrap with list()
        records = shelter.read(q)     # <-- keep your projection as-is if you have one
        if not isinstance(records, list):
            records = list(records)
    except Exception as e:
        print("CRUD read failed:", e)
        records = []

    # Keep DataTable columns/ordering stable and values JSON-safe
    return make_json_safe(records)
# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
    #FIXME Add in the code for your geolocation chart 
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can 
    # be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
            center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for 
            # the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]],
                children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]

app.run_server(debug=True)

Dash app running on http://127.0.0.1:14720/
